In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import functional as F

In [ ]:
batch_size = 100
img_height = 224
img_width = 224


In [20]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
   "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\train",
    image_size= (img_height, img_width),
    batch_size=batch_size

)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\test",
    image_size= (img_height, img_width),
    batch_size=batch_size)

val_set =  tf.keras.preprocessing.image_dataset_from_directory(
 "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\valid",


    image_size= (img_height, img_width),
    batch_size=batch_size)

Found 3130 files belonging to 2 classes.
Found 279 files belonging to 2 classes.
Found 547 files belonging to 2 classes.


In [8]:
class_names = train_set.class_names

print(class_names)

['images', 'no traffic']


In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train_set.cache().prefetch(buffer_size=AUTOTUNE)
test = test_set.cache().prefetch(buffer_size=AUTOTUNE)

img_shape = (img_height, img_width, 3)

In [3]:
import cv2

# Replace "path/to/your/image.jpg" with the actual path to your image
img = cv2.imread(r"C:\Users\sarthak mohapatra\Downloads\crash dataset - Copy\data\val\Accident\acc1 (4).jpg")

# Print the shape of the image, which includes the number of channels
print(img.shape)

(576, 720, 3)


In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False, weights='imagenet')

base_model.trainable = False
print(base_model.output_shape)

model = tf.keras.Sequential([
    base_model,
    layers.Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=32,kernel_size=(3,3) ,activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512,activation='relu'),
    layers.Dense(2, activation= 'softmax')
])
input_shape = (1,) + img_shape
model.build(input_shape)
model.summary()

(None, 7, 7, 1280)


ValueError: Computed output size would be negative. Received `inputs shape=(1, 1, 1, 32)`, `kernel shape=(3, 3, 32, 64)`, `dilation_rate=[1 1]`.

In [11]:
model.save( "jim_traffic.keras")
loaded_model = tf.keras.models.load_model(r"C:\Users\sarthak mohapatra\Desktop\vs ml  course code\jim_traffic.keras")

c:\Users\sarthak mohapatra\anaconda3\Lib\site-packages\keras\src\saving\saving_api.py:100: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr=ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbse=0.5,factor=0.5,min_lr=0.0001)
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)

In [13]:
history = model.fit(train, validation_data = val_set, epochs = 1,callbacks=[lr])

ValueError: Computed output size would be negative. Received `inputs shape=(None, 1, 1, 64)`, `kernel shape=(3, 3, 64, 64)`, `dilation_rate=[1 1]`.

In [ ]:

for i in range(len(model.weights)):
    model.weights[i]._handle_name = model.weights[i].name + "_" + str(i)
    print(model.weights[i]._handle_name)

kernel_0
gamma_1
beta_2
moving_mean_3
moving_variance_4
kernel_5
gamma_6
beta_7
moving_mean_8
moving_variance_9
kernel_10
gamma_11
beta_12
moving_mean_13
moving_variance_14
kernel_15
gamma_16
beta_17
moving_mean_18
moving_variance_19
kernel_20
gamma_21
beta_22
moving_mean_23
moving_variance_24
kernel_25
gamma_26
beta_27
moving_mean_28
moving_variance_29
kernel_30
gamma_31
beta_32
moving_mean_33
moving_variance_34
kernel_35
gamma_36
beta_37
moving_mean_38
moving_variance_39
kernel_40
gamma_41
beta_42
moving_mean_43
moving_variance_44
kernel_45
gamma_46
beta_47
moving_mean_48
moving_variance_49
kernel_50
gamma_51
beta_52
moving_mean_53
moving_variance_54
kernel_55
gamma_56
beta_57
moving_mean_58
moving_variance_59
kernel_60
gamma_61
beta_62
moving_mean_63
moving_variance_64
kernel_65
gamma_66
beta_67
moving_mean_68
moving_variance_69
kernel_70
gamma_71
beta_72
moving_mean_73
moving_variance_74
kernel_75
gamma_76
beta_77
moving_mean_78
moving_variance_79
kernel_80
gamma_81
beta_82
moving_

In [ ]:
# import os

# # Define the filename
# filename = 'existing_model.h5'

# # Check if the file exists
# if os.path.exists(filename):
#     # If it exists, remove it
#     os.remove(filename)

# # Save the model to a different filename or directory
# model.save('new_model.h5')

In [ ]:
model.save( "jim_traffic.keras")
loaded_model = tf.keras.models.load_model(r"C:\Users\sarthak mohapatra\Desktop\vs ml  course code\jim_traffic.keras")

ValueError: Exception encountered when calling Conv2D.call().

[1mKernel shape must have the same length as input, but received kernel of shape (3, 3, (1, 10, 10, 1280), 32) and input of shape [(1, 10, 10, 1280)].[0m

Arguments received by Conv2D.call():
  • args=(['<KerasTensor shape=(1, 10, 10, 1280), dtype=float32, sparse=False, name=keras_tensor_2381>'],)
  • kwargs=<class 'inspect._empty'>

In [ ]:
def predict_frame(img):
    img_array = tf.keras.utils.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction=(model.predict(img_batch) > 0.5).astype("int32")
    if(prediction[0][0]==0):
        return("no-traffic")
    else:
        return("traffic")

In [ ]:
label=[]
c=1
cap=cv2.VideoCapture(r"C:\Users\sarthak mohapatra\Downloads\15 minutes of heavy traffic noise in India _ 14-08-2022.mp4")
while True:
    ret, frame = cap.read()
    if c%30==0:
     print(c)
    #ret, frame = cap.read()
     resize_frame=cv2.resize(frame,(300,300))
  
     predict=predict_frame(resize_frame)
     label.append(predict)
    c+=1
    cv2.imshow('frame',frame)
    key=cv2.waitKey(1)&0xFF
    if  key == ord("a"):
        break
cap.release()
cv2.destroyAllWindows()

30
1/1 [==============================] - 1s 907ms/step
60
1/1 [==============================] - 0s 34ms/step
90
1/1 [==============================] - 0s 32ms/step
120
1/1 [==============================] - 0s 33ms/step
150
1/1 [==============================] - 0s 34ms/step
180
1/1 [==============================] - 0s 31ms/step
210
1/1 [==============================] - 0s 33ms/step
240
1/1 [==============================] - 0s 32ms/step
270
1/1 [==============================] - 0s 41ms/step
300
1/1 [==============================] - 0s 34ms/step
330
1/1 [==============================] - 0s 34ms/step
360
1/1 [==============================] - 0s 36ms/step
390
1/1 [==============================] - 0s 33ms/step
420
1/1 [==============================] - 0s 34ms/step
450
1/1 [==============================] - 0s 32ms/step
480
1/1 [==============================] - 0s 34ms/step
510
1/1 [==============================] - 0s 42ms/step
540
1/1 [==============================] - 0s 37ms

In [ ]:
print(label[30])

IndexError: list index out of range

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import cv2

batch_size = 100
img_height = 300
img_width = 300

train_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\train",
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\test",
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\valid",
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_set.class_names
print(class_names)

AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train_set.cache().prefetch(buffer_size=AUTOTUNE)
test = test_set.cache().prefetch(buffer_size=AUTOTUNE)

img_shape = (img_height, img_width, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
lr = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
history = model.fit(train, validation_data=val_set, epochs=1, callbacks=[lr, early_stop])

# Save the model


# Prediction function using the loaded model
def predict_frame(img):
    img_array = tf.keras.utils.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction = (loaded_model.predict(img_batch) > 0.5).astype("int32")
    if prediction[0][0] == 0:
        return "no-traffic"
    else:
        return "traffic"

label = []
c = 1
cap = cv2.VideoCapture(r"C:\Users\sarthak mohapatra\Downloads\15 minutes of heavy traffic noise in India _ 14-08-2022.mp4")
while True:
    ret, frame = cap.read()
    if c % 30 == 0:
        print(c)
    resize_frame = cv2.resize(frame, (300, 300))
    predict = predict_frame(resize_frame)
    label.append(predict)
    c += 1
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("a"):
        break
cap.release()
cv2.destroyAllWindows()
print(label[30])


Found 3130 files belonging to 2 classes.
Found 279 files belonging to 2 classes.
Found 547 files belonging to 2 classes.
['images', 'no traffic']


C:\Users\sarthak mohapatra\AppData\Local\Temp\ipykernel_15336\1514481563.py:37: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')


32/32 ━━━━━━━━━━━━━━━━━━━━ 119s 4s/step - accuracy: 0.6481 - loss: 0.6171 - val_accuracy: 0.5210 - val_loss: 0.7939 - learning_rate: 0.0010


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
model.save("jim_traffic_saved_model", save_format='tf')


ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

TensorFlow version: 2.16.1


AttributeError: module 'tensorflow.keras' has no attribute '__version__'